# Garbled Circuits

This notebook walks through Yao garbling: first a single AND gate with toy and EC-based OT, then 3-bit equality in two flavors (regular XOR with tables, and Free XOR with a global Δ).

## Ginny & Evan: single AND gate

In [2]:
:dep rand = "0.8"
:dep sha2 = "0.10"
:dep base64 = "0.21"
:dep k256 = "0.13"
:dep once_cell = "1"

use std::sync::{Mutex, MutexGuard};
use once_cell::sync::Lazy;
use rand::{rngs::StdRng, seq::SliceRandom, Rng, RngCore, SeedableRng};
use sha2::{Digest, Sha256};
use base64::{engine::general_purpose::STANDARD_NO_PAD as B64, Engine};
use k256::{elliptic_curve::{sec1::ToEncodedPoint, Field, Group}, EncodedPoint, ProjectivePoint, Scalar};

static RNG: Lazy<Mutex<StdRng>> = Lazy::new(|| Mutex::new(StdRng::seed_from_u64(1)));

fn rng() -> MutexGuard<'static, StdRng> {
    RNG.lock().expect("rng poisoned")
}

#[derive(Clone, Debug)]
struct WireLabels {
    zero: [u8; 16],
    one: [u8; 16],
}

fn rand_label() -> [u8; 16] {
    let mut bytes = [0u8; 16];
    let mut r = rng();
    r.fill_bytes(&mut bytes);
    bytes
}

fn h(k1: &[u8], k2: &[u8]) -> [u8; 32] {
    let mut hasher = Sha256::new();
    hasher.update(k1);
    hasher.update(k2);
    hasher.finalize().into()
}

fn encrypt(key: &[u8; 32], msg: &[u8; 16]) -> [u8; 16] {
    let mut out = [0u8; 16];
    for i in 0..16 {
        out[i] = msg[i] ^ key[i];
    }
    out
}

fn hash_point(pt: &ProjectivePoint) -> [u8; 32] {
    let mut hasher = Sha256::new();
    let enc: EncodedPoint = pt.to_affine().to_encoded_point(false);
    hasher.update(enc.as_bytes());
    hasher.finalize().into()
}

fn xor_label(key: &[u8; 32], label: &[u8; 16]) -> [u8; 16] {
    let mut out = [0u8; 16];
    for i in 0..16 {
        out[i] = label[i] ^ key[i];
    }
    out
}

type Cipher = [u8; 16];

Garbled AND gate:

In [3]:
struct GarbledAnd {
    table: Vec<Cipher>,
    out_labels: WireLabels,
    input_a: WireLabels,
    input_b: WireLabels,
}

// Construct one garbled AND gate from fresh labels, shuffling the table.
fn garble_and() -> GarbledAnd {
    let input_a = WireLabels {
        zero: rand_label(),
        one: rand_label(),
    };
    let input_b = WireLabels {
        zero: rand_label(),
        one: rand_label(),
    };
    let out_labels = WireLabels {
        zero: rand_label(),
        one: rand_label(),
    };

    let mut table = Vec::new();
    for a in 0..=1 {
        for b in 0..=1 {
            let key = h(
                if a == 0 { &input_a.zero } else { &input_a.one },
                if b == 0 { &input_b.zero } else { &input_b.one },
            );
            let out = if (a & b) == 1 {
                &out_labels.one
            } else {
                &out_labels.zero
            };
            table.push(encrypt(&key, out));
        }
    }

    {
        let mut r = rng();
        table.shuffle(&mut *r);
    }

    GarbledAnd {
        table,
        out_labels,
        input_a,
        input_b,
    }
}

In [4]:
// Generate one garbled AND gate instance.
let garbled = garble_and();
println!("Garbled AND generated; ready to evaluate in the next cell.");


Garbled AND generated; ready to evaluate in the next cell.


Inspect the garbled table before any party chooses inputs.


In [5]:
{
    println!("Ciphertexts (shuffled, base64) before inputs are chosen:");
    for (i, ct) in garbled.table.iter().enumerate() {
        println!("  row {i}: `{}`", B64.encode(ct));
    }

    println!("\nInput labels (base64) known to both parties (via OT/pre-share):");
    println!("  Ginny g=0: `{}`", B64.encode(garbled.input_a.zero));
    println!("  Ginny g=1: `{}`", B64.encode(garbled.input_a.one));
    println!("  Evan  e=0: `{}`", B64.encode(garbled.input_b.zero));
    println!("  Evan  e=1: `{}`", B64.encode(garbled.input_b.one));
}


Ciphertexts (shuffled, base64) before inputs are chosen:


  row 0: `BSluLup0quDMeWNH8/glMA`


  row 1: `0yVtZqtH/mXzbNli2tK5iQ`


  row 2: `FWNvNZ6z2eIS+e4b1/3Nsg`


  row 3: `JEnqXL82rT+XDltbLrew+g`


...

  Ginny g=0: `YRgw02QaaPlKaQ3MJdH0sA`


  Ginny g=1: `2slIMlrBj23TJWQ3FzXzLA`


  Evan  e=0: `VHQ9xeJNKkE7FZWEwSJt9g`


  Evan  e=1: `7M/EyqjNe2PHy1b/Zs1gtQ`


()

Ginny now chooses a label; we assume a secure, authenticated channel when she sends it to Evan.

In [6]:
let ginny_pref: u8 = {
    let mut r = rng();
    if r.gen_bool(0.5) { 1 } else { 0 }
}; // Ginny decides now
let ginny_label: [u8; 16] = if ginny_pref == 0 {
    garbled.input_a.zero
} else {
    garbled.input_a.one
};

// Secure, authenticated channel is assumed; send Ginny's chosen label directly.
println!("Ginny preference g={ginny_pref}; sending label over the secure channel:");
println!("  label (base64): `{}`", B64.encode(ginny_label));
println!("Evan receives the correct Ginny label over the secure channel.");

Ginny preference g=0; sending label over the secure channel:


  label (base64): `YRgw02QaaPlKaQ3MJdH0sA`


Evan receives the correct Ginny label over the secure channel.


Oblivious transfer (toy demo; real protocols hide Evan’s choice and the unchosen label). Evan inputs his bit `e`; Ginny holds both Evan labels and uses OT to deliver only the chosen one.


In [7]:
fn toy_ot(sender_l0: &[u8; 16], sender_l1: &[u8; 16], choice: u8) -> [u8; 16] {
    // Toy/semi-honest only: just returns the chosen label; does not hide choice or the other label.
    if choice == 0 {
        *sender_l0
    } else {
        *sender_l1
    }
}

// Evan picks his bit and obtains the matching label via the toy OT.
let evan_pref: u8 = {
    let mut r = rng();
    if r.gen_bool(0.5) { 1 } else { 0 }
};
let evan_label = toy_ot(&garbled.input_b.zero, &garbled.input_b.one, evan_pref);
println!("Evan preference e={evan_pref}; OT delivers his input label:");
println!("  label (base64): `{}`", B64.encode(evan_label));

Evan preference e=0; OT delivers his input label:


  label (base64): `VHQ9xeJNKkE7FZWEwSJt9g`


Oblivious transfer (toy demo; real protocols hide Evan’s choice and the unchosen label). Evan inputs his bit `e`; Ginny holds both Evan labels and uses OT to deliver only the chosen one. **This toy OT leaks Evan’s choice to Ginny**, but assume she learns nothing here—we’ll introduce a proper, secure OT later.

In [8]:
// Evan evaluates once using his label and Ginny's label; keep the output label/bit for sending back.
let mut evan_output_label: Option<[u8; 16]> = None;
let mut evan_output_bit: Option<u8> = None;
{
    let key = h(&ginny_label, &evan_label);
    for ct in &garbled.table {
        let out = encrypt(&key, ct);
        if out == garbled.out_labels.zero {
            evan_output_label = Some(out);
            evan_output_bit = Some(0);
            break;
        }
        if out == garbled.out_labels.one {
            evan_output_label = Some(out);
            evan_output_bit = Some(1);
            break;
        }
    }
}
let evan_output_label = evan_output_label.expect("no matching row decrypted");
let evan_output_bit = evan_output_bit.expect("no matching bit decrypted");
println!("Recovered AND result: g={} ∧ e={} -> {}", ginny_pref, evan_pref, evan_output_bit);


Recovered AND result: g=0 ∧ e=0 -> 0


Evan sends the decrypted output label back to Ginny.


In [9]:
// Evan sends the output label to Ginny (no re-computation).
println!("Evan sends output label to Ginny (base64): `{}`", B64.encode(evan_output_label));


Evan sends output label to Ginny (base64): `prOzHBw/NYXqiOUnqBmwYg`


Ginny maps the returned label to a bit using her output-label map.


In [10]:
// Ginny decodes the label she received.
{
    let ginny_decoded = if evan_output_label == garbled.out_labels.zero { 0 } else { 1 };
    println!("Ginny maps the label to bit: {}", ginny_decoded);
    assert_eq!(ginny_decoded, evan_output_bit);
}


Ginny maps the label to bit: 0


()

### Simple secure 1-of-2 OT (secp256k1 + SHA-256)
Let's revisit our toy OT and use something more secure. We now split the steps into explicit sender/receiver phases for one bit.

Sender/receiver summary: the sender holds both labels; the receiver picks one bit.

Secure OT flow steps:
1) Sender samples an offset `D = d·G` and publishes it.
2) Receiver samples a keypair, embeds their choice into `(pk, pk + D)`, and sends those to the sender.
3) Sender encrypts both labels to `(pk, pk + D)` using fresh randomness, returning two ciphertexts with their `R` points.
4) Receiver decrypts only the chosen ciphertext using their secret key. The sender cannot tell which one was chosen.

This remains semi-honest and uses only secp256k1 ECDH plus SHA-256; no OT extension is needed for a single bit.

In [11]:
// Deterministic OT helpers using seeded RNG
#[derive(Clone, Debug)]
struct SenderOffset {
    point: ProjectivePoint,
    bytes: EncodedPoint,
}

#[derive(Clone, Debug)]
struct SenderCiphertexts {
    r0: ProjectivePoint,
    c0: [u8; 16],
    r1: ProjectivePoint,
    c1: [u8; 16],
}

#[derive(Clone, Debug)]
struct ReceiverState {
    sk: Scalar,
    choice: u8,
}

fn ot_sender_offset() -> SenderOffset {
    // Sender picks D = d·G once per OT and publishes it.
    let mut r = rng();
    let d = Scalar::random(&mut *r);
    drop(r);
    let point = ProjectivePoint::GENERATOR * &d;
    let bytes: EncodedPoint = point.to_affine().to_encoded_point(false);
    SenderOffset { point, bytes }
}

fn ot_receiver_choose(
    d_point: &ProjectivePoint,
    choice: u8,
    ) -> (ReceiverState, ProjectivePoint, ProjectivePoint, EncodedPoint, EncodedPoint) {
    // Receiver picks a keypair and embeds the choice bit into (pk, pk_plus_d).
    let g = ProjectivePoint::GENERATOR;
    let mut r = rng();
    let sk = Scalar::random(&mut *r);
    drop(r);
    let pk = g * &sk;
    let (pk_base, pk_plus_d) = if choice == 0 { (pk, pk + d_point) } else { (pk + d_point, pk) };
    let pk_base_bytes: EncodedPoint = pk_base.to_affine().to_encoded_point(false);
    let pk_plus_d_bytes: EncodedPoint = pk_plus_d.to_affine().to_encoded_point(false);
    (
        ReceiverState { sk, choice },
        pk_base,
        pk_plus_d,
        pk_base_bytes,
        pk_plus_d_bytes,
    )
}

fn ot_sender_encrypt(
    label0: [u8; 16],
    label1: [u8; 16],
    pk_base: &ProjectivePoint,
    pk_plus_d: &ProjectivePoint,
    ) -> SenderCiphertexts {
    // Sender encrypts both labels to the two public keys using fresh randomness.
    let g = ProjectivePoint::GENERATOR;

    let mut r = rng();
    let r0_s = Scalar::random(&mut *r);
    let r1_s = Scalar::random(&mut *r);
    drop(r);

    let r0 = g * &r0_s;
    let k0 = hash_point(&(pk_base * &r0_s));
    let c0 = xor_label(&k0, &label0);

    let r1 = g * &r1_s;
    let k1 = hash_point(&(pk_plus_d * &r1_s));
    let c1 = xor_label(&k1, &label1);

    SenderCiphertexts { r0, c0, r1, c1 }
}

fn ot_receiver_decrypt(state: &ReceiverState, cts: &SenderCiphertexts) -> [u8; 16] {
    // Receiver can decrypt only the chosen ciphertext using their secret key.
    let (r, c) = if state.choice == 0 { (&cts.r0, &cts.c0) } else { (&cts.r1, &cts.c1) };
    let k = hash_point(&(r * &state.sk));
    xor_label(&k, c)
}

Ginny (the sender) cannot see Evan’s (the receiver) choice, and Evan learns only one label. Ginny did create both labels, so she knows which raw value is `e=0` vs `e=1`, but OT hides which one Evan actually receives—as long as he never shows her his input label.

**Demo steps:** we run the secure OT in four messages and print each intermediate object. The receiver’s choice stays hidden, but we show the public data for teaching.

Step 1 (sender): publish a fresh random offset `D = d·G` and send it to the receiver. This point is public and used to embed the choice later.

In [12]:
let offset = ot_sender_offset();
let d_point = offset.point.clone();
println!("Sender offset D (uncompressed, base64): `{}`", B64.encode(offset.bytes.as_bytes()));

Sender offset D (uncompressed, base64): `BIUIX5SqFWLbtn2sNdGv8MbZmcc+HAQEoWz7SwJSJPaa6gZDLHlYiUp8mQnOK2yMh4oiePxUuXMa+/KIfVw7Z7k`


Step 2 (receiver): sample a fresh keypair `(sk, pk)`, derive `pk + D`, and send both keys in an order that encodes the choice bit: choice 0 → `(pk, pk + D)`, choice 1 → `(pk + D, pk)`. Ginny sees two ordinary public keys and cannot tell which one was shifted. We print both public keys in uncompressed base64.

Cheating note: if Evan sent `(pk, pk)`, he could decrypt both. A malicious-secure OT variant would add a consistency check (e.g., an OR-proof that either `pk1 - pk0 = D` or `pk0 - pk1 = D`) to stop that, but this demo stays semi-honest.

**Ginny’s view (what the sender knows/sees):**

```
D = d·G (known to Ginny)
|
| publishes to Evan
v
{ pk , pk + D }  (unordered; Ginny can’t tell which is shifted)
  |         |
  |         +-- encrypt -> (R1, c1)
  +------------ encrypt -> (R0, c0)
```

- Public offset: `D = d·G` (she also knows the secret scalar `d`).
- Two receiver points: an unordered pair `{pk, pk + D}` (she cannot tell which is shifted).
- Her own labels: `(label0, label1)` for Evan.
- What she does *not* know: Evan’s choice bit, Evan’s secret key `sk`, or which point is the base vs shifted.
- What she sends: two ciphertext tuples `(R0, c0)` to one point and `(R1, c1)` to the other, derived independently with fresh randomness.

She knows d and D, sees the two points, but not which is base vs shifted, and never sees sk or the choice.

**Evan’s view (this run):**

Receiver choice bit (printed above): e = 0 → he chose the base key `pk`.

```
Ginny sent ciphertexts bound to the two public keys:
  pk      -> (R0, c0)
  pk + D  -> (R1, c1)

Evan uses his one secret key sk (no +D applied by him):
  k = H(sk · R0); decrypts c0 -> label
  c1 stays garbage
```

Because e=0 in this run, only `(R0, c0)` decrypts; `(R1, c1)` remains unintelligible.

In [13]:
// Step 2 (receiver): embed choice into (pk, pk_plus_d) and send to sender.
let (receiver_state, pk, pk_plus_d, _, _) = ot_receiver_choose(&d_point, evan_pref);
let pk_bytes: EncodedPoint = pk.to_affine().to_encoded_point(false);
let pk_plus_d_bytes: EncodedPoint = pk_plus_d.to_affine().to_encoded_point(false);
println!("Receiver choice bit e={evan_pref}");
println!("pk (base, uncompressed, base64): `{}`", B64.encode(pk_bytes.as_bytes()));
println!("pk_plus_d (shifted, uncompressed, base64): `{}`", B64.encode(pk_plus_d_bytes.as_bytes()));

Receiver choice bit e=0


pk (base, uncompressed, base64): `BIUIX5SqFWLbtn2sNdGv8MbZmcc+HAQEoWz7SwJSJPaa6gZDLHlYiUp8mQnOK2yMh4oiePxUuXMa+/KIfVw7Z7k`


pk_plus_d (shifted, uncompressed, base64): `BIVGfSBUa5HzS1P4GXsPgt81gvvvV4YuQOcbJ3xblYQFXZ+P5A9Hqr8xYgPlZuJOGadyjMAhG6U4/Hc/T5T+FoA`


Step 3 (sender): encrypt both labels to `(pk, pk + D)` using fresh per-ciphertext randomness `r0_s`/`r1_s`. These are ephemeral DH secrets to derive one-time keys (R = r·G, k = H(pk·r)), keeping each ciphertext unlinkable even if the receiver reuses their keypair. We print the two ciphertexts and their R values (uncompressed base64).

In [14]:
// Step 3 (sender): encrypt both labels to (pk, pk_plus_d).
let sender_cts = {
    let cts = ot_sender_encrypt(garbled.input_b.zero, garbled.input_b.one, &pk, &pk_plus_d);
    let r0_bytes: EncodedPoint = cts.r0.to_affine().to_encoded_point(false);
    let r1_bytes: EncodedPoint = cts.r1.to_affine().to_encoded_point(false);

    let r0_b64 = B64.encode(r0_bytes.as_bytes());
    let r1_b64 = B64.encode(r1_bytes.as_bytes());
    let c0_b64 = B64.encode(cts.c0);
    let c1_b64 = B64.encode(cts.c1);
    let short = |s: &str, n: usize| s.chars().take(n).collect::<String>();

    println!("Cipher for pk (e=0): R={}..., c={}...", short(&r0_b64, 24), short(&c0_b64, 16));
    println!("Cipher for pk_plus_d (e=1): R={}..., c={}...", short(&r1_b64, 24), short(&c1_b64, 16));
    cts
};

Cipher for pk (e=0): R=BIUIX5SqFWLbtn2sNdGv8MbZ..., c=6lbvV51qaGUEqz9T...


Cipher for pk_plus_d (e=1): R=BBF2OT1f7Ha2gTOGthdoCXYF..., c=NhbJs3CJw0/wNFOq...


Step 4 (receiver): decrypt only the chosen ciphertext with the receiver’s secret key.


In [15]:
// Step 4 (receiver): decrypt only the chosen ciphertext.
let evan_secure_label = ot_receiver_decrypt(&receiver_state, &sender_cts);
println!("Receiver recovers label for e={evan_pref} (base64, keep private): `{}`", B64.encode(evan_secure_label));
assert_eq!(evan_secure_label, if evan_pref == 0 { garbled.input_b.zero } else { garbled.input_b.one });

Receiver recovers label for e=0 (base64, keep private): `VHQ9xeJNKkE7FZWEwSJt9g`


For larger circuits, if you only need a handful of bits and speed is not a concern, just run the four-step OT (offset, choose, encrypt, decrypt) separately for each bit.
- Fresh randomness each time; do not reuse keys or offsets across runs.
- Cost scales linearly with the number of bits (one public-key OT per bit), but privacy and correctness stay the same under the semi-honest assumption.
- OT extension only matters when you need many OTs and want to amortize the public-key work with fast symmetric crypto.

### Optional: OR-proof to stop `(pk, pk)` cheating

For the purpose of understanding BitVM3 we don't need this. The following is completely unverified.

Idea: receiver proves that the submitted pair encodes a valid shift by the published offset `D`, without revealing which key is shifted. Statements:
- S₀: `pk1 - pk0 = D` (pk0 base, pk1 shifted)
- S₁: `pk0 - pk1 = D` (pk1 base, pk0 shifted)

Use a standard Σ-protocol OR-proof from Chaum–Pedersen equality-of-discrete-log proofs, made non-interactive with Fiat–Shamir:
1) Build a real CP transcript for the true statement; simulate the other.
2) Hash all public values to one challenge; split it across the two branches (real + simulated).
3) Output both transcripts. Verifier checks both; soundness under DDH, zero-knowledge via simulation.

Cost: a few group elements + scalars. This notebook stays semi-honest; add this only if you need malicious security.

References: Chaum–Pedersen (Eurocrypt ’92); Cramer–Damgård–Schoenmakers OR-proofs (CRYPTO ’94); Fiat–Shamir (1986) for NIZK via hashing.

### 3-bit equality: regular XOR vs Free XOR
Alice and Bob each hold a secret 3-bit number. They only want to learn if the numbers are identical. Bob receives his three input labels via the naive OT (called three times). We show two versions of the same circuit:
- **Regular garbling:** every XOR has a 4-row table, then two OR gates reduce the three bitwise differences.
- **Free XOR:** the XORs cost nothing; only the OR gates keep tables.

Regular circuit (3 XOR tables + 2 OR tables):
```
a0 ─⊕─┐
       ├─≥1?─┐
a1 ─⊕─┘       │
        OR    ├─≥1?── diff_any (0 = equal, 1 = differ)
a2 ─⊕─┐       │
       └─⊕────┘
```
Free XOR circuit (XORs are label XORs; only ORs are garbled):
```
a0 ─⊕free─┐
          ├─≥1?─┐
a1 ─⊕free─┘      │
           OR    ├─≥1?── diff_any
a2 ─⊕free─┐      │
          └─⊕free┘
```

In [16]:
// Helpers shared by both equality demos
fn xor16(a: &[u8; 16], b: &[u8; 16]) -> [u8; 16] {
    let mut out = [0u8; 16];
    for i in 0..16 {
        out[i] = a[i] ^ b[i];
    }
    out
}

fn fmt_bits(bits: &[u8; 3]) -> String {
    bits.iter().map(|b| b.to_string()).collect::<Vec<_>>().join("")
}

#[derive(Clone, Debug)]
struct BinaryGate {
    table: Vec<Cipher>,
    out: WireLabels,
    in_a: WireLabels,
    in_b: WireLabels,
}

fn garble_binary_gate_with_labels<F>(in_a: WireLabels, in_b: WireLabels, out: WireLabels, truth: F) -> BinaryGate
where
    F: Fn(u8, u8) -> u8,
{
    let mut table = Vec::new();
    for a in 0..=1 {
        for b in 0..=1 {
            let key = h(
                if a == 0 { &in_a.zero } else { &in_a.one },
                if b == 0 { &in_b.zero } else { &in_b.one },
            );
            let out_label = if truth(a, b) == 1 { &out.one } else { &out.zero };
            table.push(encrypt(&key, out_label));
        }
    }
    {
        let mut r = rng();
        table.shuffle(&mut *r);
    }
    BinaryGate { table, out, in_a, in_b }
}

fn garble_binary_gate<F>(truth: F) -> BinaryGate
where
    F: Fn(u8, u8) -> u8,
{
    let in_a = WireLabels { zero: rand_label(), one: rand_label() };
    let in_b = WireLabels { zero: rand_label(), one: rand_label() };
    let out = WireLabels { zero: rand_label(), one: rand_label() };
    garble_binary_gate_with_labels(in_a, in_b, out, truth)
}

fn eval_gate(gate: &BinaryGate, la: &[u8; 16], lb: &[u8; 16]) -> Option<([u8; 16], u8)> {
    let key = h(la, lb);
    for ct in &gate.table {
        let out = encrypt(&key, ct);
        if out == gate.out.zero {
            return Some((out, 0));
        }
        if out == gate.out.one {
            return Some((out, 1));
        }
    }
    None
}

#### Run 1A: regular garbling — build & show tables

We garble three XOR gates and two OR gates, print the shuffled tables, and keep everything for evaluation. Alice and Bob reuse the shared 3-bit secrets chosen once above; Bob’s labels would arrive via three toy OTs.

#### Shared secrets for both equality runs

Alice and Bob draw one 3-bit secret each; both the regular and Free XOR scenarios reuse these bits instead of resampling per run.

In [17]:
// Draw Alice and Bob's 3-bit secrets once for both scenarios, deterministically to avoid churn between runs
let mut seeded = StdRng::seed_from_u64(42);
let alice_bits_shared: [u8; 3] = [
    seeded.gen_bool(0.5) as u8,
    seeded.gen_bool(0.5) as u8,
    seeded.gen_bool(0.5) as u8,
];
let bob_bits_shared: [u8; 3] = [
    seeded.gen_bool(0.5) as u8,
    seeded.gen_bool(0.5) as u8,
    seeded.gen_bool(0.5) as u8,
];

println!("Alice (A): {} (binary)", fmt_bits(&alice_bits_shared));
println!("Bob   (B): {} (binary)", fmt_bits(&bob_bits_shared));

Alice (A): 000 (binary)


Bob   (B): 111 (binary)


In [18]:
println!("=== Regular garbled XOR equality (3 bits) — build & show tables ===");

fn show_gate(name: &str, gate: &BinaryGate) {
    println!("{name} table ({} ciphertexts):", gate.table.len());
    for (i, ct) in gate.table.iter().enumerate() {
        println!("  row {i}: {}", B64.encode(ct));
    }
}

let alice_bits_eq = alice_bits_shared;
let bob_bits_eq = bob_bits_shared;
println!("Alice (A): {} (binary)", fmt_bits(&alice_bits_eq));
println!("Bob   (B): {} (binary)", fmt_bits(&bob_bits_eq));

let xor_gates: [BinaryGate; 3] = [
    garble_binary_gate(|a, b| a ^ b),
    garble_binary_gate(|a, b| a ^ b),
    garble_binary_gate(|a, b| a ^ b),
];
let or_gate_01 = garble_binary_gate_with_labels(
    xor_gates[0].out.clone(),
    xor_gates[1].out.clone(),
    WireLabels { zero: rand_label(), one: rand_label() },
    |a, b| a | b,
    );
let or_gate_any = garble_binary_gate_with_labels(
    or_gate_01.out.clone(),
    xor_gates[2].out.clone(),
    WireLabels { zero: rand_label(), one: rand_label() },
    |a, b| a | b,
    );

show_gate("XOR0", &xor_gates[0]);
show_gate("XOR1", &xor_gates[1]);
show_gate("XOR2", &xor_gates[2]);
show_gate("OR(xor0,xor1)", &or_gate_01);
show_gate("OR(diff01,xor2)", &or_gate_any);

=== Regular garbled XOR equality (3 bits) — build & show tables ===


Alice (A): 000 (binary)


Bob   (B): 111 (binary)


XOR0 table (4 ciphertexts):


  row 0: ZRqVuPs/LVb6NdAJPmh6XQ


...

OR(diff01,xor2) table (4 ciphertexts):


  row 0: zWY1PSAe8TFi+SZCybLIGw


  row 1: koR6Phw/FIDIysHvBfLCPg


  row 2: T6ZuX8ZJAJ1uPH20s60olg


  row 3: O1gwd/uElxGzz0n16Jx4bw


#### Run 1B: regular garbling — evaluate
Reuse the tables from Run 1A, feed Alice’s bits locally, pretend Bob’s labels arrived via three toy OTs, and decode the final equality bit (1 = equal, 0 = different).

In [19]:
println!("=== Regular garbled XOR equality (3 bits) — evaluate ===");

fn pick_label(bit: u8, labels: &WireLabels) -> [u8; 16] {
    if bit == 0 { labels.zero } else { labels.one }
}

let alice_labels: [[u8; 16]; 3] = [
    pick_label(alice_bits_eq[0], &xor_gates[0].in_a),
    pick_label(alice_bits_eq[1], &xor_gates[1].in_a),
    pick_label(alice_bits_eq[2], &xor_gates[2].in_a),
];
let bob_labels: [[u8; 16]; 3] = [
    pick_label(bob_bits_eq[0], &xor_gates[0].in_b),
    pick_label(bob_bits_eq[1], &xor_gates[1].in_b),
    pick_label(bob_bits_eq[2], &xor_gates[2].in_b),
];

let (diff0_lbl, diff0_bit) = eval_gate(&xor_gates[0], &alice_labels[0], &bob_labels[0]).expect("xor0 eval");
let (diff1_lbl, diff1_bit) = eval_gate(&xor_gates[1], &alice_labels[1], &bob_labels[1]).expect("xor1 eval");
let (diff2_lbl, diff2_bit) = eval_gate(&xor_gates[2], &alice_labels[2], &bob_labels[2]).expect("xor2 eval");

let (diff01_lbl, diff01_bit) = eval_gate(&or_gate_01, &diff0_lbl, &diff1_lbl).expect("or01 eval");
let (anydiff_lbl, anydiff_bit) = eval_gate(&or_gate_any, &diff01_lbl, &diff2_lbl).expect("or-any eval");

let decoded_anydiff = if anydiff_lbl == or_gate_any.out.zero {
    0
} else if anydiff_lbl == or_gate_any.out.one {
    1
} else {
    panic!("unexpected label on final wire");
};
assert_eq!(decoded_anydiff, anydiff_bit);

let eq_bit = if anydiff_bit == 0 { 1u8 } else { 0u8 };
println!("equality result: {}", eq_bit);

=== Regular garbled XOR equality (3 bits) — evaluate ===


equality result: 0


#### Free XOR primer (before Run 2A)

Free XOR assigns each wire a base label `W0`; its `W1` is just `W0 ⊕ Δ` for a circuit-wide offset `Δ` known only to the garbler. XOR gates then cost nothing because the labels themselves already carry the bit:

- Bit 0 on a wire: evaluator holds `W0`.

- Bit 1 on a wire: evaluator holds `W0 ⊕ Δ`.


Concrete combinations at an XOR gate (inputs A,B with their actual labels):

- (0,0): `A0 ⊕ B0 = W0(A) ⊕ W0(B)` → output should be bit 0, label `W0(out)`.

- (1,1): `(W0(A) ⊕ Δ) ⊕ (W0(B) ⊕ Δ) = W0(A) ⊕ W0(B)` (Δ cancels) → also bit 0, same `W0(out)`.

- (1,0): `(W0(A) ⊕ Δ) ⊕ W0(B) = W0(A) ⊕ W0(B) ⊕ Δ` → bit 1, label `W1(out)`.

- (0,1): `W0(A) ⊕ (W0(B) ⊕ Δ) = W0(A) ⊕ W0(B) ⊕ Δ` → bit 1, same `W1(out)`.

So the evaluator just XORs the two input labels; Δ either cancels (for equal bits) or remains once (for differing bits), matching the correct output label without a table.


For this notebook we choose `Δ` deterministically: hash a fixed tag to 16 bytes and force the low bit to 1 so `Δ ≠ 0`, which guarantees `W0 ≠ W1` on every wire. `Δ` is reused across all wires in the circuit, never revealed to the evaluator, and never derived per wire. The label values stay pseudorandom to anyone without `Δ` and the wire seeds.

In [20]:
// Free XOR helpers introduced just before the Free XOR run
static DELTA: Lazy<[u8; 16]> = Lazy::new(|| {
    // Deterministic for the notebook: hash a fixed tag, then force LSB=1
    let mut hasher = Sha256::new();
    hasher.update(b"free-xor-delta");
    let digest: [u8; 32] = hasher.finalize().into();
    let mut d = [0u8; 16];
    d.copy_from_slice(&digest[..16]);
    d[0] |= 1; // ensure non-zero so W0 != W1
    d
});

#[derive(Clone, Debug)]
struct FreeWire {
    zero: [u8; 16],
}

impl FreeWire {
    fn one(&self) -> [u8; 16] { xor16(&self.zero, &*DELTA) }
}

fn new_free_wire() -> FreeWire {
    FreeWire { zero: rand_label() }
}

fn free_labels(w: &FreeWire) -> WireLabels {
    WireLabels { zero: w.zero, one: w.one() }
}

fn free_xor_wire(a: &FreeWire, b: &FreeWire) -> FreeWire {
    FreeWire { zero: xor16(&a.zero, &b.zero) }
}

#### Run 2A: Free XOR — build & show tables

Free XOR makes the XOR gates table-free; we only garble the two OR gates. Alice and Bob reuse the same 3-bit secrets sampled once above; we print the OR tables.

In [21]:
println!("=== Free XOR equality (3 bits) — build & show tables ===");

let alice_free_bits = alice_bits_shared;
let bob_free_bits = bob_bits_shared;

let a_wires: [FreeWire; 3] = [new_free_wire(), new_free_wire(), new_free_wire()];
let b_wires: [FreeWire; 3] = [new_free_wire(), new_free_wire(), new_free_wire()];

let diff_wires: [FreeWire; 3] = [
    free_xor_wire(&a_wires[0], &b_wires[0]),
    free_xor_wire(&a_wires[1], &b_wires[1]),
    free_xor_wire(&a_wires[2], &b_wires[2]),
];

let or01_out = new_free_wire();
let or_any_out = new_free_wire();

let or_free_01 = garble_binary_gate_with_labels(
    free_labels(&diff_wires[0]),
    free_labels(&diff_wires[1]),
    free_labels(&or01_out),
    |a, b| a | b,
);
let or_free_any = garble_binary_gate_with_labels(
    free_labels(&or01_out),
    free_labels(&diff_wires[2]),
    free_labels(&or_any_out),
    |a, b| a | b,
    );

show_gate("OR_free(diff0,diff1)", &or_free_01);
show_gate("OR_free(diff01,diff2)", &or_free_any);

=== Free XOR equality (3 bits) — build & show tables ===


OR_free(diff0,diff1) table (4 ciphertexts):


  row 0: k2vLSmcPOru6K/1NGK+dnA


  row 1: gRrQkIWXwuA9caTbTADx1w


  row 2: DzV7yDpl1gCfA1J4Fst93w


...

OR_free(diff01,diff2) table (4 ciphertexts):


  row 0: KMnGnUIW6fhV+aojUgEPNg


  row 1: YOjgTKdEjhzbImL3MjTSuw


  row 2: DvepaK1ceBfknye9aWv7bg


  row 3: /rh9vn16b9oU7tQVx8UN5w


#### Run 2B: Free XOR — evaluate
Use the free XOR wiring to compute the three diff labels without tables, then pass through the two OR tables, decode, and report equality.

In [22]:
println!("=== Free XOR equality (3 bits) — evaluate ===");

fn label_with_delta(zero: &[u8; 16], delta: &[u8; 16], bit: u8) -> [u8; 16] {
    if bit == 0 { *zero } else { xor16(zero, delta) }
}

fn decode_with_delta(zero: &[u8; 16], delta: &[u8; 16], lbl: &[u8; 16]) -> Option<u8> {
    if *lbl == *zero { Some(0) } else if *lbl == xor16(zero, delta) { Some(1) } else { None }
}

let delta = xor16(&or_free_01.in_a.zero, &or_free_01.in_a.one);

let alice_free_labels: [[u8; 16]; 3] = [
    label_with_delta(&a_wires[0].zero, &delta, alice_free_bits[0]),
    label_with_delta(&a_wires[1].zero, &delta, alice_free_bits[1]),
    label_with_delta(&a_wires[2].zero, &delta, alice_free_bits[2]),
];
let bob_free_labels: [[u8; 16]; 3] = [
    label_with_delta(&b_wires[0].zero, &delta, bob_free_bits[0]),
    label_with_delta(&b_wires[1].zero, &delta, bob_free_bits[1]),
    label_with_delta(&b_wires[2].zero, &delta, bob_free_bits[2]),
];

let diff0_lbl = xor16(&alice_free_labels[0], &bob_free_labels[0]);
let diff1_lbl = xor16(&alice_free_labels[1], &bob_free_labels[1]);
let diff2_lbl = xor16(&alice_free_labels[2], &bob_free_labels[2]);

let diff0_bit = decode_with_delta(&diff_wires[0].zero, &delta, &diff0_lbl).expect("decode diff0");
let diff1_bit = decode_with_delta(&diff_wires[1].zero, &delta, &diff1_lbl).expect("decode diff1");
let diff2_bit = decode_with_delta(&diff_wires[2].zero, &delta, &diff2_lbl).expect("decode diff2");

let (diff01_lbl, diff01_bit) = eval_gate(&or_free_01, &diff0_lbl, &diff1_lbl).expect("or_free_01 eval");
let (anydiff_lbl, anydiff_bit) = eval_gate(&or_free_any, &diff01_lbl, &diff2_lbl).expect("or_free_any eval");

let decoded_anydiff = decode_with_delta(&or_any_out.zero, &delta, &anydiff_lbl).expect("decode final");
assert_eq!(decoded_anydiff, anydiff_bit);

let eq_bit = if anydiff_bit == 0 { 1u8 } else { 0u8 };
println!("equality result: {}", eq_bit);

=== Free XOR equality (3 bits) — evaluate ===


equality result: 0
